# AutoCron Demo 2: Advanced Features

This notebook demonstrates advanced scheduling features.

## Features Covered:
- Retries and error handling
- Timeouts
- Task priorities
- Task dependencies
- Task metadata and tags

## Setup

In [8]:
from autocron import AutoCron, schedule
from datetime import datetime
import time
import random

## 1. Retries and Error Handling

Tasks can automatically retry on failure.

In [10]:
attempt_count = 0

@schedule(every='10s', retries=3, retry_delay=5)
def unreliable_api_call():
    """Simulates an unreliable API that might fail."""
    global attempt_count
    attempt_count += 1

    print(f"[{datetime.now().strftime('%H:%M:%S')}] Attempt #{attempt_count}")

    if random.random() < 0.6:
        print("API call failed")
        raise Exception("Connection timeout")

    print("API call succeeded")
    return "Data retrieved"

print("Task with retries scheduled")
print("Will retry up to 3 times with 5-second delay")

2026-02-22 15:38:15 - INFO - Task 'unreliable_api_call' scheduled with: interval=10s


Task with retries scheduled
Will retry up to 3 times with 5-second delay


In [11]:
scheduler = AutoCron()

def flaky_task():
    if random.random() < 0.5:
        raise Exception("Random failure")
    return "Success"

scheduler.add_task(
    name="flaky_task",
    func=flaky_task,
    every='15s',
    retries=5,
    retry_delay=3
)

print("Task added with 5 retries and 3-second delay")

2026-02-22 15:38:19 - INFO - Task 'flaky_task' scheduled with: interval=15s


Task added with 5 retries and 3-second delay


## 2. Timeouts

Prevent tasks from running too long.

In [12]:
@schedule(every='20s', timeout=10)
def long_running_task():
    """Task that might take too long."""
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Starting long task")

    duration = random.randint(5, 15)
    print(f"Processing for {duration} seconds")
    time.sleep(duration)

    print("Task completed")
    return "Done"

print("Task with 10-second timeout scheduled")
print("If task takes longer than 10 seconds, it will be terminated")

2026-02-22 15:38:22 - INFO - Task 'long_running_task' scheduled with: interval=20s


Task with 10-second timeout scheduled
If task takes longer than 10 seconds, it will be terminated


In [13]:
scheduler = AutoCron()

def slow_processing():
    time.sleep(random.randint(3, 8))
    return "Processed"

scheduler.add_task(
    name="slow_processor",
    func=slow_processing,
    every='30s',
    timeout=5,
    retries=2,
    retry_delay=2
)

print("Task with timeout and retries configured")

2026-02-22 15:38:25 - INFO - Task 'slow_processor' scheduled with: interval=30s


Task with timeout and retries configured


## 3. Task Priorities

Control execution order when multiple tasks are ready.

In [14]:
scheduler = AutoCron()

def critical_task():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] CRITICAL: Security check")

def high_priority_task():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] HIGH: Data backup")

def normal_task():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] NORMAL: Log rotation")

def low_priority_task():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] LOW: Cache cleanup")

priority_map = {
    "critical": 10,
    "high": 7,
    "normal": 5,
    "low": 2,
}

scheduler.add_task(name="critical", func=critical_task, every='10s')
scheduler.add_task(name="high", func=high_priority_task, every='10s')
scheduler.add_task(name="normal", func=normal_task, every='10s')
scheduler.add_task(name="low", func=low_priority_task, every='10s')

print("Tasks scheduled")
print("Priority map:", priority_map)

2026-02-22 15:38:28 - INFO - Task 'critical' scheduled with: interval=10s
2026-02-22 15:38:28 - INFO - Task 'high' scheduled with: interval=10s
2026-02-22 15:38:28 - INFO - Task 'normal' scheduled with: interval=10s
2026-02-22 15:38:28 - INFO - Task 'low' scheduled with: interval=10s


Tasks scheduled
Priority map: {'critical': 10, 'high': 7, 'normal': 5, 'low': 2}


## 4. Task Dependencies

Run tasks only after other tasks complete.

In [15]:
scheduler = AutoCron()

def fetch_data():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Step 1: Fetching raw data")
    time.sleep(2)
    return "raw_data.csv"

def process_data():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Step 2: Processing data")
    time.sleep(2)
    return "processed_data.csv"

def generate_report():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Step 3: Generating report")
    time.sleep(1)
    return "report.pdf"

scheduler.add_task(name="fetch", func=fetch_data, every='1h')
scheduler.add_task(name="process", func=process_data, every='1h')
scheduler.add_task(name="report", func=generate_report, every='1h')

print("Task pipeline scheduled")
print("Pipeline order is managed in task logic: fetch -> process -> report")

2026-02-22 15:38:32 - INFO - Task 'fetch' scheduled with: interval=1h
2026-02-22 15:38:32 - INFO - Task 'process' scheduled with: interval=1h
2026-02-22 15:38:32 - INFO - Task 'report' scheduled with: interval=1h


Task pipeline scheduled
Pipeline order is managed in task logic: fetch -> process -> report


## 5. Task Metadata and Tags

Organize and categorize tasks with metadata and tags.

In [16]:
scheduler = AutoCron()

task_catalog = {
    "db_backup": {
        "tags": ["backup", "database", "critical"],
        "metadata": {
            "owner": "DevOps Team",
            "database": "production",
            "retention_days": 30,
            "notification_email": "ops@company.com",
        },
    },
    "log_cleanup": {
        "tags": ["maintenance", "cleanup"],
        "metadata": {
            "owner": "Platform Team",
            "cleanup_age_days": 7,
            "disk_threshold_gb": 100,
        },
    },
    "api_monitor": {
        "tags": ["monitoring", "api", "health-check"],
        "metadata": {
            "owner": "Engineering Team",
            "endpoints": ["api.example.com", "api2.example.com"],
            "alert_threshold_ms": 500,
        },
    },
}

scheduler.add_task(name="db_backup", func=lambda: print("Backing up database"), every='6h')
scheduler.add_task(name="log_cleanup", func=lambda: print("Cleaning old logs"), every='1d')
scheduler.add_task(name="api_monitor", func=lambda: print("Monitoring API health"), every='5m')

print("Tasks with external metadata catalog added\n")
for task in scheduler.list_tasks():
    info = task_catalog.get(task.name, {})
    print(f"Task: {task.name}")
    print(f"Tags: {', '.join(info.get('tags', []))}")
    print(f"Owner: {info.get('metadata', {}).get('owner', 'N/A')}")
    print()

2026-02-22 15:38:35 - INFO - Task 'db_backup' scheduled with: interval=6h
2026-02-22 15:38:35 - INFO - Task 'log_cleanup' scheduled with: interval=1d
2026-02-22 15:38:35 - INFO - Task 'api_monitor' scheduled with: interval=5m


Tasks with external metadata catalog added

Task: db_backup
Tags: backup, database, critical
Owner: DevOps Team

Task: log_cleanup
Tags: maintenance, cleanup
Owner: Platform Team

Task: api_monitor
Tags: monitoring, api, health-check
Owner: Engineering Team



## 6. Max Instances Control

Prevent multiple instances of the same task from running simultaneously.

In [17]:
scheduler = AutoCron()

def heavy_processing():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Starting heavy processing")
    time.sleep(10)
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Processing complete")

scheduler.add_task(
    name="heavy_task",
    func=heavy_processing,
    every='5s',
)

print("Task configured")
print("This AutoCron version does not expose max_instances; long-running overlap control must be implemented in task logic")

2026-02-22 15:38:40 - INFO - Task 'heavy_task' scheduled with: interval=5s


Task configured
This AutoCron version does not expose max_instances; long-running overlap control must be implemented in task logic


## 7. Task Enable/Disable

Temporarily disable tasks without removing them.

In [18]:
scheduler = AutoCron()

active_task_id = scheduler.add_task(
    name="active_task",
    func=lambda: print("Running"),
    every='10s',
)

disabled_task_id = scheduler.add_task(
    name="disabled_task",
    func=lambda: print("This will not run while disabled"),
    every='10s',
)

disabled_task = scheduler.get_task(task_id=disabled_task_id)
if disabled_task is not None:
    disabled_task.enabled = False

print("Tasks added\n")
for task in scheduler.list_tasks():
    status = "ENABLED" if task.enabled else "DISABLED"
    print(f"{status}: {task.name}")

2026-02-22 15:38:43 - INFO - Task 'active_task' scheduled with: interval=10s
2026-02-22 15:38:43 - INFO - Task 'disabled_task' scheduled with: interval=10s


Tasks added

ENABLED: active_task
DISABLED: disabled_task


## 8. Real-World Example: Complete ETL Pipeline

Combining multiple advanced features in a realistic scenario.

In [19]:
scheduler = AutoCron()

def extract_data():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Extracting data from API")
    time.sleep(2)
    if random.random() < 0.2:
        raise Exception("API connection failed")
    return "raw_data"

def transform_data():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Transforming data")
    time.sleep(3)
    return "processed_data"

def load_data():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Loading data to database")
    time.sleep(2)
    return "success"

task_catalog = {
    "extract": {"owner": "Data Team", "pipeline_step": 1},
    "transform": {"owner": "Data Team", "pipeline_step": 2},
    "load": {"owner": "Data Team", "pipeline_step": 3},
}

scheduler.add_task(
    name="extract",
    func=extract_data,
    every='1h',
    retries=3,
    retry_delay=10,
    timeout=30,
)

scheduler.add_task(
    name="transform",
    func=transform_data,
    every='1h',
    timeout=60,
)

scheduler.add_task(
    name="load",
    func=load_data,
    every='1h',
    timeout=30,
)

print("Complete ETL pipeline configured\n")
print("Pipeline sequence: extract -> transform -> load")
print("External task catalog:", task_catalog)

2026-02-22 15:38:46 - INFO - Task 'extract' scheduled with: interval=1h
2026-02-22 15:38:46 - INFO - Task 'transform' scheduled with: interval=1h
2026-02-22 15:38:46 - INFO - Task 'load' scheduled with: interval=1h


Complete ETL pipeline configured

Pipeline sequence: extract -> transform -> load
External task catalog: {'extract': {'owner': 'Data Team', 'pipeline_step': 1}, 'transform': {'owner': 'Data Team', 'pipeline_step': 2}, 'load': {'owner': 'Data Team', 'pipeline_step': 3}}


## Summary

In this demo, you learned:

 **Retries** - Automatic retry with configurable delays

 **Timeouts** - Prevent tasks from running too long

 **Priorities** - Control execution order (1-10 scale)

 **Dependencies** - Create task pipelines

 **Metadata & Tags** - Organize and categorize tasks

 **Max Instances** - Prevent concurrent executions

 **Enable/Disable** - Toggle tasks on/off

### Next Steps:
- Check out `03_async_tasks.ipynb` for async/await support
- See `04_persistence.ipynb` for saving and loading tasks
- Explore `05_safe_mode.ipynb` for secure task execution